In [9]:
import os
import pandas as pd

from datetime import datetime

## printing all best results

In [1]:
metrics = {
    "cola": ["mcc"],
    "sst2": ["accuracy"],
    "mrpc": ["f1", "accuracy"],
    "stsb": ["pearson"],
    "qqp": ["f1", "accuracy"],
    "mnli_matched": ["accuracy"],
    "mnli_mismatched": ["accuracy"],
    "qnli": ["accuracy"],
    "rte": ["accuracy"],
    "wnli": ["accuracy"],
}

In [12]:
def parse_results_name(filename):
    _, date, time, model = filename.split(".")[0].split("_")

    date_obj = datetime.strptime(date + time, '%Y%m%d%H%M%S')

    formatted_date = date_obj.strftime('%Y-%m-%d')
    formatted_time = date_obj.strftime('%H:%M:%S')

    return formatted_date, formatted_time, model

In [23]:
results_path = "results"

for root, dirs, files in os.walk(results_path):
    for file in files:
        if not file.endswith("csv"):
            continue

        df = pd.read_csv(os.path.join(root, file))

        # skipping empty results files
        if df.shape[0] == 0:
            continue

        _, embedding_type, task = root.split("/")
        date, time, model = parse_results_name(file)
        print(f"--- Results from {date} {time}: {model}, {embedding_type}, {task} ---")

        for m in metrics[task]:
            best = df[m].max()
            best_row = df[df[m] == best]
            print(f"Best {m}: {best:.5f}")

        

--- Results from 2023-11-29 15:53:54: DistilRoBERTaBase, mean_pooling, cola ---
Best mcc: 0.49233
--- Results from 2023-12-08 01:11:58: MPNetBase, mean_pooling, cola ---
Best mcc: 0.43973
--- Results from 2023-12-13 17:34:56: MPNetBase, mean_pooling, cola ---
Best mcc: 0.46715
--- Results from 2023-11-29 15:58:59: DistilRoBERTaBase, mean_pooling, mrpc ---
Best f1: 0.72304
Best accuracy: 0.72304
--- Results from 2023-12-01 15:35:54: DistilRoBERTaBase, mean_pooling, mrpc ---
Best f1: 0.72059
Best accuracy: 0.72059
--- Results from 2023-11-29 16:02:59: DistilRoBERTaBase, mean_pooling, rte ---
Best accuracy: 0.57040
--- Results from 2023-12-13 17:55:22: MPNetBase, mean_pooling, rte ---
Best accuracy: 0.61372
--- Results from 2023-11-29 16:03:58: DistilRoBERTaBase, mean_pooling, wnli ---
Best accuracy: 0.56338
--- Results from 2023-12-13 17:56:02: MPNetBase, mean_pooling, wnli ---
Best accuracy: 0.56338
--- Results from 2023-11-29 16:21:24: DistilRoBERTaBase, mean_pooling, qnli ---
Best acc